In [ ]:
import zmq
import numpy as np
import time
import matplotlib.pyplot as plt
from xmlrpc.client import ServerProxy

In [ ]:
# Launch the companion gnuradio flow hackrf_spectrum_analyzer.grc 
# or hackrf_spectrum_analyzer-high_resolution.grc before running this notebook

In [ ]:
xmlrpc_control_client = ServerProxy('http://'+'localhost'+':18080')

xmlrpc_control_client.set_f_tx_MHz(152)

In [ ]:
def get_spectrum(ip = '192.168.86.33', port = 55555):
    # Retrieve the current frequency spectrum from the gnuradio flow
    context = zmq.Context()
    socket = context.socket(zmq.SUB)
    socket.connect("tcp://%s:%d"%(ip,port)) # connect, not bind, the PUB will bind,>
    socket.setsockopt(zmq.SUBSCRIBE, b'') # subscribe to topic of all (needed or el>
    fft_len = xmlrpc_control_client.get_fft_len()
    freq = xmlrpc_control_client.get_freq()
    samp_rate = xmlrpc_control_client.get_samp_rate()
    fstart = (freq-samp_rate/2)
    fstep = (samp_rate/fft_len)
    f_Hz = np.arange(fft_len)*fstep+fstart
    while True:
        if socket.poll(10) != 0: # check if there is a message on the socket
            msg = socket.recv() # grab the message
            data = np.frombuffer(msg, dtype=np.float32, count=-1)
            if len(data) == fft_len:
                socket.close()
                break
        else:
            time.sleep(0.1) # wait 100ms and try again
    return f_Hz, data

In [ ]:
f_Hz,data = get_spectrum()

In [ ]:
plt.figure()
plt.plot(f_Hz/1e6,data)
plt.xlabel("Frequency [MHz]")
plt.ylabel("Power")